# Package Install
```
$ sudo apt-get install python2.7-dev
$ sudo easy_install pip
$ pip install jupyter
$ pip install pandas
$ pip install google-api-python-client
```

# Jupyter spark enviroment.
```
export PYSPARK_DRIVER_PYTHON=jupyter
export PYSPARK_DRIVER_PYTHON_OPTS="notebook"
pyspark --master local[2] --packages graphframes:graphframes:0.3.0-spark2.0-s_2.11
```

In [4]:
import pandas as pd

projectId = 'spark-151209'
query = """SELECT Actor1CountryCode, Actor2CountryCode, EventRootCode FROM [gdelt-bq:full.events] 
WHERE Actor1CountryCode != ""
AND Actor2CountryCode != ""
AND EventRootCode = '19'
LIMIT 100000"""
df = pd.read_gbq(query, projectId)

Requesting query... ok.
Query running...
Query done.
Processed: 3.7 Gb

Retrieving results...
  Got page: 1; 100.0% done. Elapsed 7.02 s.
Got 100000 rows.

Total time taken 7.65 s.
Finished at 2017-01-12 09:08:35.


In [5]:
sqlctx = SQLContext(sc)
sdf = sqlctx.createDataFrame(df)
sdf.show()

+-----------------+-----------------+-------------+
|Actor1CountryCode|Actor2CountryCode|EventRootCode|
+-----------------+-----------------+-------------+
|              USA|              USA|           19|
|              USA|              NZL|           19|
|              USA|              USA|           19|
|              CAN|              POL|           19|
|              USA|              USA|           19|
|              USA|              USA|           19|
|              ZAF|              ZAF|           19|
|              GBR|              USA|           19|
|              USA|              USA|           19|
|              USA|              USA|           19|
|              USA|              USA|           19|
|              BGD|              BGD|           19|
|              USA|              USA|           19|
|              NZL|              NZL|           19|
|              UKR|              POL|           19|
|              SAU|              ARE|           19|
|           

In [6]:
import graphframes


In [ ]:
keys = sdf.rdd.flatMap(lambda x: (x[0], x[1])).distinct()
keylist = keys.collect()
vertices = keys.map(lambda x: (x,)).toDF(["id"])
edge = sdf.rdd.map(lambda x: (x[0], x[1], x[0] + ":" +x[1])).toDF(["src", "dst", "relation"])


In [ ]:
g = graphframes.GraphFrame(vertices, edge)
results = g.pageRank(resetProbability=0.15, tol=0.01)

In [ ]:
text = results.vertices.sort('pagerank', ascending=False).select("id", "pagerank").show()
type(text)